In [1]:
import sys
sys.path.append(r'C:\Users\USER\Desktop\spark-kurs\Projekt\loaders')

from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

import os
os.environ['PYSPARK_HOME']= r"C:\Users\USER\Desktop\spark-kurs\Projekt"
os.environ['PYSPARK_DRIVER_PYTHON']= "jupyter"
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
os.environ['PYSPARK_PYTHON'] = 'python'

from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, FloatType
from pyspark.sql.functions import col, split, explode, sha2, round, avg, udf

from tweets_loader import TweetsLoader

spark = SparkSession.builder \
        .appName("projekt") \
        .config("spark.executor.memory", "2g") \
        .config("spark.driver.memory", "2g") \
        .master("local[*]") \
        .getOrCreate()

In [4]:
tweets_loader = TweetsLoader()

all_tweets_df = tweets_loader.load_all_tweets({
    "covid": r"C:\Users\USER\Desktop\spark-kurs\Projekt\data\covid19_tweets.csv",
    "grammys": r"C:\Users\USER\Desktop\spark-kurs\Projekt\data\GRAMMYs_tweets.csv",
    "financial": r"C:\Users\USER\Desktop\spark-kurs\Projekt\data\financial.csv"
})

all_tweets_df.show()
all_tweets_df.printSchema()

+--------------------+--------------------+--------------------+-------------------+--------------+------------+---------------+-------------+-------------------+--------------------+--------------------+-------------------+----------+--------+----+---------+-------+-------------+----+--------+
|           user_name|       user_location|    user_description|       user_created|user_followers|user_friends|user_favourites|user_verified|               date|                text|            hashtags|             source|is_retweet|category|  id|timestamp|symbols|company_names| url|verified|
+--------------------+--------------------+--------------------+-------------------+--------------+------------+---------------+-------------+-------------------+--------------------+--------------------+-------------------+----------+--------+----+---------+-------+-------------+----+--------+
|     Time4fisticuffs|    Pewee Valley, KY|#Christian #Catho...|2009-02-28 18:57:41|          9275|        9525|